# Scenarios and Levels specified in the national plan

| **Scenario(s)** | **Trigger Type**               | **Alert Level** | **Description**                                                                 |
|-----------------|-------------------------------|------------------|---------------------------------------------------------------------------------|
| 1 & 3           | Clinical & Epidemiological    | Level 0         | Safe zone + slight increase in diarrhea cases                                  |
| 1 & 3           | Clinical & Epidemiological    | Level 1         | Alert zone + <30% adult cases + rumors of community increase                   |
| 1 & 3           | Clinical & Epidemiological    | Level 2         | Epidemic zone + ≥30% adult cases + community cases or deaths                   |
| 1 & 3           | Clinical & Epidemiological    | Level 3         | Positive RDT + Positive culture + ≥10% hospitalization rate                    |
| 2               | Climate Event                 | Level 0         | Weather alert for above-average rainfall or flood risk                         |
| 2               | Climate Event                 | Level 1         | Disaster alert + risk to water/sanitation in cholera hotspots                  |
| 2               | Climate Event                 | Level 2         | Disaster occurred + ≥30% adult cases or community deaths                       |
| 2               | Humanitarian Crisis           | Level 1         | Armed attacks in cholera hotspot areas                                         |
| 2               | Humanitarian Crisis           | Level 2         | Armed attacks + displacement + ≥30% adult cases or community deaths            |


This notebook looks at the increase in cases by district and how often the 30% increase is recorded.

In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [4]:
# Data and visualization
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path
import os
import re

# Azure Blob Storage
from azure.storage.blob import ContainerClient, BlobClient
import io

# Display settings
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.0f}".format
sns.set(style="whitegrid")

In [5]:
postos_path = (
    Path(os.getenv("AA_DATA_DIR"))
    / "private"
    / "raw"
    / "moz"
    / "cholera"
    / "DISTRIBUIÇÃO DAS ZONAS QUENTES E DE ALTO RISCO DE CÓLERA NO PAÍS.xlsx"
)

Consolidating the worksheets in the file into one table.


In [ ]:
postos_risk_all = pd.read_excel(postos_path, sheet_name=None, header=None)
postos_dfs = list(postos_risk_all.values())

# Read first sheet again with headers to get column names
postos_risk = pd.read_excel(postos_path, sheet_name="Table042 (Page 38)")
columns = postos_risk.columns

# Process remaining sheets: skip first row, assign headers
for df in postos_dfs[1:]:
    df = df.iloc[1:]
    df.columns = columns
    postos_risk = pd.concat([postos_risk, df], ignore_index=True)

# Remove rows with NaN in 'provincia'
postos_risk = postos_risk[postos_risk["Província"].notna()]
# Remove \n from all string values
for col in postos_risk.select_dtypes(include="object"):
    postos_risk[col] = postos_risk[col].map(
        lambda x: re.sub(r"\s+", " ", x).strip() if isinstance(x, str) else x
    )
# Standardize column names (lowercase, replace spaces and special characters with underscores)
postos_risk.columns = (
    postos_risk.columns.str.lower()
    .str.replace(r"\s+", "_", regex=True)
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

In [24]:
postos_risk

,cod,provncia,distrito,carga_clera,carga_diarreia,unidade_administrativa_posto_administrativo_bairro,classificao_de_risco_de_clera,populao_exposta_2023,incidncia_mdia_anual_da_clera,incidncia_mdia_anual_da_diarreia
1,MACC2_01,Cabo Delgado,Pemba,9,7,Cariacó,Zona Quente,"58,721",158,"4,172"
2,MACC2_02,Cabo Delgado,Pemba,9,7,Ingonane,Zona Quente,"22,050",48,"2,175"
3,MACC2_03,Cabo Delgado,Pemba,9,7,Natite,Zona Quente,"39,602",62,"5,493"
4,MACC2_04,Cabo Delgado,Pemba,9,7,Cimento,Zona Quente,"19,670",87,"26,432"
5,MACC2_05,Cabo Delgado,Pemba,9,7,Eduardo Mondlane,Zona Quente,"25,458",331,"4,990"
...,...,...,...,...,...,...,...,...,...,...
306,MACC2_302,Cabo Delgado,Balama,1,3,Balama-Sede,Zona de Alto Risco,108071,-,585
307,MACC2_303,Cabo Delgado,Balama,1,3,Mavala,Zona de Alto Risco,32577,-,230
308,MACC2_304,Cabo Delgado,Muidumbe,0,3,Muidumbe Sede-/ Mua-tide,Zona de Alto Risco,13021,-,"5,896"
309,MACC2_305,Cabo Delgado,Muidumbe,0,3,Chitunda,Zona de Alto Risco,42600,-,"3,235"


In [ ]:
postos_risk.to_csv(
    Path(os.getenv("AA_DATA_DIR"))
    / "private"
    / "processed"
    / "moz"
    / "cholera"
    / "postos_risk_levels.csv",
    index=False,
    encoding="utf-8",
    quotechar='"',
    quoting=1,
)

Some logic: If the 